In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Cargar los datos de CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocesar los datos
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalización
y_train = to_categorical(y_train, 10)  # One-hot encoding
y_test = to_categorical(y_test, 10)

# Definir un bloque residual
def residual_block(x, filters, kernel_size=3, stride=1):
    # Guardar la entrada para la conexión residual
    res = x

    # Primera capa convolucional
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Segunda capa convolucional
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Ajustar la dimensión de la entrada a la del bloque (si es necesario)
    if res.shape[-1] != filters:
        res = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(res)
        res = layers.BatchNormalization()(res)

    # Añadir la conexión residual (skip connection)
    x = layers.add([x, res])
    x = layers.ReLU()(x)

    return x

# Definir la arquitectura de la red
input_layer = layers.Input(shape=(32, 32, 3))  # CIFAR-10 tiene imágenes de 32x32x3

x = layers.Conv2D(64, kernel_size=3, strides=1, padding='same')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Agregar bloques residuales
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 128)
x = residual_block(x, 128)

# Capa de agrupamiento global
x = layers.GlobalAveragePooling2D()(x)

# Capa densa para clasificación
x = layers.Dense(10, activation='softmax')(x)

# Crear el modelo
model = models.Model(inputs=input_layer, outputs=x)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluar el modelo
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 32, 32, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 32, 32, 64)     │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 32, 32, 64)     │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 32, 32, 64)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 32, 32, 64)     │         36,928 │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 32, 32, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 32, 32, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 32, 32, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 32, 32, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 32, 32, 64)     │              0 │ batch_normalization_2… │
│                           │                        │                │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 32, 32, 64)     │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 32, 32, 64)     │         36,928 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 32, 32, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 32, 32, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 32, 32, 64)     │         36,928 │ re_lu_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 32, 32, 64)     │            256 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)          

 Total params: 679,562 (2.59 MB)

 Trainable params: 677,642 (2.58 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 69ms/step - accuracy: 0.4309 - loss: 1.5598 - val_accuracy: 0.4049 - val_loss: 1.7909
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 58ms/step - accuracy: 0.6398 - loss: 1.0033 - val_accuracy: 0.5589 - val_loss: 1.2132
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 85s 61ms/step - accuracy: 0.7015 - loss: 0.8423 - val_accuracy: 0.6990 - val_loss: 0.8472
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.7498 - loss: 0.7207 - val_accuracy: 0.5149 - val_loss: 1.7680
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.7827 - loss: 0.6244 - val_accuracy: 0.6735 - val_loss: 0.9394
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.8073 - loss: 0.5642 - val_accuracy: 0.6324 - val_loss: 1.1299
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 58ms/step - accuracy: 0.8310 - loss: 0.4937 - val_accuracy: 0.6785 - val_loss: 0.9580
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 84s 61ms/step - accuracy: 0.8432 - loss: 0.4535 - 